✍ В предыдущем юните мы разобрали ансамблевый метод под названием бустинг, а также рассмотрели различные его вариации. В этом юните мы решим практический кейс с использованием данного алгоритма, а также сравним изученные модификации бустинга.

Мы будем работать c данными, которые содержат результаты опроса клиентов авиакомпании по поводу их удовлетворённости полётом.

Нашей задачей будет предсказать удовлетворённость пассажиров.

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn import tree
from sklearn import ensemble
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
df = pd.read_csv("data/AirPass.csv", index_col=0)
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


+ Gender — пол пассажира (женский, мужской);
+ Customer Type — тип клиента (постоянный/непостоянный клиент);
+ Age — возраст клиента;
+ Type of Travel — цель перелета (личная/деловая поездка);
+ Class — туристический класс пассажира (Business, Eco, Eco Plus);
+ Flight distance — расстояние полета;
+ Inflight wifi service — уровень удовлетворённости Wi-Fi (0 — не применимо, 1–5);
+ Departure/Arrival time convenient — уровень удовлетворённости временем отправления и прибытия;
+ Ease of Online booking — уровень удовлетворённости онлайн-бронированием;
+ Gate location — уровень удовлетворённости расположением выхода на посадку;
+ Food and drink — уровень удовлетворённости едой и напитками;
+ Online boarding — уровень удовлетворённости онлайн-регистрацией;
+ Seat comfort — уровень удовлетворённости комфортом сидений;
+ Inflight entertainment — уровень удовлетворённости развлечениями на борту;
+ On-board service — уровень удовлетворённости сервисом на борту;
+ Leg room service — уровень удовлетворённости местом для ног;
+ Baggage handling — уровень удовлетворённости обработкой багажа;
+ Check-in service — уровень удовлетворённости услугами регистрации;
+ Inflight service — уровень удовлетворённости обслуживанием во время полёта;
+ Cleanliness — уровень удовлетворённости чистотой;
+ Departure Delay in Minutes — задержка при отправлении (в минутах);
+ Arrival Delay in Minutes — задержка при прибытии (в минутах);
+ **Satisfaction** — удовлетворённость авиакомпанией — *целевая переменная (satisfaction/neutral/dissatisfaction)*.

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  object 
 2   Customer Type                      103904 non-null  object 
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  object 
 5   Class                              103904 non-null  object 
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink                     1039

## Задание 6.1

Для начала сделаем небольшую предобработку данных. Сколько всего в данных пропущенных значений?

In [34]:
df.isnull().sum().sum()

310

## Задание 6.2

Теперь давайте избавимся от найденных пропусков. Заполните их все медианными значениями. После этого вычислите среднее арифметическое для признака, отражающего задержку при прибытии в минутах. Ответ округлите до двух знаков после точки-разделителя.

In [35]:
df["Arrival Delay in Minutes"].fillna(df["Arrival Delay in Minutes"].median(), inplace=True)
df["Arrival Delay in Minutes"].mean()

15.133392362180475

## Задание 6.3

Проведём небольшой разведывательный анализ. Посмотрим, в каких категориях пассажиров превалировали удовлетворённые полетом клиенты.



In [36]:
df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0]

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: satisfaction, dtype: float64

In [37]:
df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0]

Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: satisfaction, dtype: float64

In [38]:
df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0]

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: satisfaction, dtype: float64

In [39]:
# Перекодируем часть бинарных признаков, чтобы использовать их при обучении:
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

## Задание 6.4

Для остальных категориальных признаков создайте dummy-переменные. Сделайте это с помощью функции get_dummies() из библиотеки Pandas, параметры не меняйте. Сколько теперь признаков в данных (включая целевую переменную)?

In [40]:
df = pd.get_dummies(df)
df.shape

(103904, 26)

## Задание 6.5

Мы практически добрались до обучения модели. Разбейте данные на обучающую и тестовую выборки в соотношении 80/20, параметр random_state = 26. Сколько наблюдений попало в тестовую выборку?

In [41]:
X = df.drop(columns='satisfaction')
y = df["satisfaction"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

## Задание 6.6

Теперь нам необходимо реализовать масштабирование данных. Для этого обучите на обучающей выборке метод StandardScaler() и с помощью него преобразуйте и обучающую, и тестовую выборки. Не забудьте, что целевую переменную обрабатывать не нужно.

*Примечание. Отметим, что если бы дальше мы работали только с деревьями, масштабирование бы не требовалось. Однако мы реализуем его, чтобы можно было обучать и другие модели и сравнивать полученные результаты.*

В качестве ответа введите самое первое значение из матрицы преобразованных признаков тестовой выборки. Округлите значение до двух знаков после точки-разделителя.

In [42]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

0.9408251379303

## Задание 6.7

Перейдём к обучению моделей. В качестве первой модели возьмём самую простую — логистическую регрессию. Мы делаем это для того, чтобы потом сравнивать с ней полученные результаты: так вы сможете выяснить, насколько ансамбли смогут улучшить точность прогноза.

Обучите логистическую регрессию с параметрами по умолчанию на наших данных. В качестве ответа введите значение метрики f1_score. Ответ округлите до трёх знаков после точки-разделителя.

In [43]:
from sklearn.metrics import f1_score

#Создаём объект класса LogisticRegression
log_reg = linear_model.LogisticRegression()

#Обучаем модель 
log_reg.fit(X_train, y_train)
#Делаем предсказание класса
preds_test = log_reg.predict(X_test)
f1_score(preds_test, y_test)

0.8546883773161146

## Задание 6.8

Теперь перейдём к бустингу. Начнём с обучения первой модели — AdaBoost. В качестве базовой модели для неё возьмите решающее дерево с параметром random_state = 26.

Обучите AdaBoost, зафиксировав random_state со значением 26 и задав темп обучения 0.01. В качестве ответа введите значение метрики f1_score. Ответ округлите до трёх знаков после точки-разделителя.

In [44]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

model_ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=26),random_state=26,learning_rate=0.01)

model_ada.fit(X_train, y_train)
preds_test = model_ada.predict(X_test)
f1_score(preds_test, y_test)

0.9404794558121674

## Задание 6.9

Перейдем к следующему алгоритму — градиентному бустингу.

Будем настраивать количество деревьев и темп обучения, делая перебор по следующей сетке:

`params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}`

Используйте для поиска оптимальных параметров GridSearchCV, а для ускорения работы алгоритма задайте параметр кросс-валидации, равный 3.

Какое наибольшее значение метрики f1_score получилось? Ответ округлите до трёх знаков после точки-разделителя.

*Примечание. Необходимо указать лучший результат в методе GridSearchCV на тренировочных данных.*

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

model_gs = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = GridSearchCV(model_gs, 
                  params, 
                  cv=3, 
                  scoring='f1',
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.9s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.7s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.6s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.6s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.6s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.3s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   0.6s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.6s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   0.6s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.2s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.2s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

## Задание 6.10

Обучите алгоритм XGBoost. Так как он достаточно мощный «из коробки», определите его с параметрами по умолчанию, только задайте random_state = 26. Какое значение метрики f1_score получилось? Ответ округлите до трёх знаков после точки-разделителя.

In [47]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train,y_train)
preds_test = model_xgb.predict(X_test)
f1_score(preds_test, y_test)

0.9579785161685312

## Задание 6.11

Обучите алгоритм CatBoost. Как и XGBoost, будем обучать его с настройками по умолчанию и заданным random_state = 26. Какое значение метрики f1_score получилось? Ответ округлите до трёх знаков после точки-разделителя.

In [49]:
from catboost import CatBoostClassifier, Pool

model_cb = CatBoostClassifier(random_state=26)
model_cb.fit(X_train, y_train)
preds_class = model_cb.predict(X_test)
f1_score(preds_class, y_test)

Learning rate set to 0.068023
0:	learn: 0.6018110	total: 150ms	remaining: 2m 30s
1:	learn: 0.5020759	total: 218ms	remaining: 1m 48s
2:	learn: 0.4472436	total: 254ms	remaining: 1m 24s
3:	learn: 0.4028708	total: 285ms	remaining: 1m 10s
4:	learn: 0.3674776	total: 317ms	remaining: 1m 3s
5:	learn: 0.3397880	total: 382ms	remaining: 1m 3s
6:	learn: 0.3121199	total: 510ms	remaining: 1m 12s
7:	learn: 0.2917489	total: 577ms	remaining: 1m 11s
8:	learn: 0.2749032	total: 610ms	remaining: 1m 7s
9:	learn: 0.2575190	total: 641ms	remaining: 1m 3s
10:	learn: 0.2473699	total: 673ms	remaining: 1m
11:	learn: 0.2377539	total: 717ms	remaining: 59s
12:	learn: 0.2279313	total: 797ms	remaining: 1m
13:	learn: 0.2212511	total: 896ms	remaining: 1m 3s
14:	learn: 0.2100359	total: 1.02s	remaining: 1m 6s
15:	learn: 0.2025731	total: 1.09s	remaining: 1m 6s
16:	learn: 0.1942298	total: 1.12s	remaining: 1m 4s
17:	learn: 0.1877932	total: 1.2s	remaining: 1m 5s
18:	learn: 0.1832383	total: 1.24s	remaining: 1m 3s
19:	learn: 0.1

0.9598221728257851

## Задание 6.12

Выведите матрицу ошибок для алгоритма, который получил наилучшие показатели качества модели на обучающей выборке (будем считать, что оцениваем по f1_score). Матрица ошибок выводится в следующем формате:

In [52]:
from catboost.utils import get_confusion_matrix

cm = get_confusion_matrix(model_cb, data=Pool(X_train,y_train), thread_count=-1)
print(cm)

[[46668.   538.]
 [ 1255. 34662.]]


## Задание 6.13

Оцените важность признаков для модели из предыдущего задания. Отметьте признак, который оказывает наибольшее влияние на значение целевой переменной:

In [58]:
pd.DataFrame(
    {
        "feature_importance": model_cb.get_feature_importance(),
        "feature_names": df.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)

,feature_importance,feature_names
6,25.260967,Inflight wifi service
4,18.532405,Type of Travel
2,7.373765,Customer Type
11,7.331015,Online boarding
22,5.520554,Class_Business
17,3.864015,Checkin service
3,3.791656,Age
16,3.586867,Baggage handling
9,3.283935,Gate location
12,2.971666,Seat comfort


## ⭐ **Поздравляем**, вы справились с достаточно сложной задачей и добились высокого качества выделения. Советуем не останавливаться на достигнутом и попробовать организовать наиболее посещаемый ансамбль. Тем не менее, будьте готовы: перебор гиперпараметров на большие объёмы данных занимает достаточно много времени.

Уже в следующем юните вы изучите ещё один вид ансамблей и пополните свой арсенал алгоритмов →